## 4.2. Process land cover based on UMEP Input format

In [ ]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
import os
import geopandas as gpd


UPSCALE_FACTOR = 10 # scale from 10m -> 1m
INPUT_RASTER = "/media/remap/NO_HEAT_RB/City_Atlanta/Raw/LULC/2024_dw_composite_raw.tif"
RESAMPLED_RASTER_PATH = "/media/remap/NO_HEAT_RB/City_Atlanta/Processed/resampled_lulc.tif"
BUILDING_POLYGON = "/media/remap/NO_HEAT_RB/City_Atlanta/Processed/Building_Footprints/building_ft_height.geojson"
OUTPUT_RASTER = "/media/remap/NO_HEAT_RB/City_Atlanta/Processed/LULC/LULC_umep.tif"

In [ ]:
os.makedirs(os.path.dirname(RESAMPLED_RASTER_PATH), exist_ok=True)
os.makedirs(os.path.dirname(OUTPUT_RASTER),exist_ok=True)


# 1) Resample with nearest‐neighbor (for categorical LULC)
with rasterio.open(INPUT_RASTER) as src:
    profile     = src.profile.copy()
    new_height  = src.height * UPSCALE_FACTOR
    new_width   = src.width  * UPSCALE_FACTOR
    new_transform = src.transform * src.transform.scale(
        (src.width  / new_width),
        (src.height / new_height)
    )
    profile.update({
        'height':    new_height,
        'width':     new_width,
        'transform': new_transform
    })
    data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.nearest
    )
    with rasterio.open(RESAMPLED_RASTER_PATH, 'w', **profile) as dst:
        dst.write(data)

# 2) Reclassify on the *resampled* raster

vector = gpd.read_file(BUILDING_POLYGON)
with rasterio.open(RESAMPLED_RASTER_PATH) as src:
    profile = src.profile.copy()    
    profile.update(nodata=0)  # set LULC no‑data to 0
    transform = src.transform
    crs = src.crs

    # Reproject vector if needed
    if vector.crs != crs:
        vector = vector.to_crs(crs)

    geom = [(geom, 2) for geom in vector.geometry]

    # Rasterize full building layer
    building_layer = rasterize(
        geom,
        out_shape=(src.height, src.width),
        transform=transform,
        fill=0,
        all_touched=False,
        merge_alg=rasterio.enums.MergeAlg.replace,
        dtype=np.int16
    )

    with rasterio.open(OUTPUT_RASTER, 'w', **profile) as dst: # no data as 0 in LULC
        for ji, window in src.block_windows(1):  # 1 is the band index
            data = src.read(1, window=window)
            original = data.copy()
            
            data = np.full_like(original, fill_value=0)
            
            data[np.isin(original, [1, 2, 3, 4, 5])] = 5  # grass
            data[original == 0] = 7                      # water
            data[original == 6] = 2                      # roof
            data[original == 7] = 6                      # baresoil
            data[original == 8] = 0                      # nodata   
            building_window = building_layer[
                window.row_off : window.row_off + window.height,
                window.col_off : window.col_off + window.width
            ]
            data = np.where(building_window == 2, 2, data)
            
            dst.write(data, window=window, indexes=1)

print(f"Land Cover Process Done in {OUTPUT_RASTER}")